In [ ]:
pip install contractions

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import contractions
import re
import string
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt
import nltk
nltk.download("all")
from nltk.corpus import stopwords as sw
from wordcloud import WordCloud, STOPWORDS


In [ ]:
blog_details = pd.read_csv('../input/ctdsmedium-blogs/sb_blog.csv', encoding='ISO-8859-1', parse_dates=['release_date'])
blog_details

In [ ]:
print("Category vs Claps: ", blog_details.groupby('category')['claps'].sum())
print("\nRatio of Women-to-Men heroes is ",blog_details.groupby('heroes_gender')['heroes'].count()[0]/(blog_details.groupby('heroes_gender')['heroes'].count()[0] + blog_details.groupby('heroes_gender')['heroes'].count()[1]))

In [ ]:
# Sorted by claps in descending order 
sorted_blog_details = blog_details.sort_values('claps', ascending=False)
plt.bar(sorted_blog_details['heroes'], sorted_blog_details['claps'])
plt.xlabel('Heroes', fontsize=15)
plt.ylabel('Claps', fontsize=15)
plt.xticks(sorted_blog_details['heroes'], fontsize=9, rotation=90)
plt.title('Heroes Vs Claps')
plt.figure(figsize=(20,15))
plt.show()

In [ ]:
#pre processing methods
def clean_the_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    # Expanding Contractions eg. i've to I have
    text = contractions.fix(text)
    text = re.sub(r".*native.*\n?","",text)    
    text = re.sub('Hey.+science.', '', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
          
    text = re.sub(r".*Audio.*\n?","",text)    
    text = re.sub(r".*sanyambhutani.com.*\n?","",text)
    text = re.sub(r".*Newsletter.*\n?","",text)
    text = re.sub(r".*so much.*\n?","",text)
    text = re.sub(r".*Sanyam Bhutani:.*\n?","",text)
    text = re.sub(r".*Subscribe.*\n?","",text)
    text = re.sub(r".*twitter.*\n?","",text)
    text = re.sub(r".*Flow.*\n?","",text)
    text = re.sub(r".*Follow.*\n?","",text)
    text = re.sub(r".*Playlist.*\n?","",text)
    text = re.sub(r".*support.*\n?","",text)
    text = re.sub(r".*Linkedin.*\n?","",text)
    text = re.sub(r".*Kaggle:.*\n?","",text)
    text = re.sub(r".*hosted.*\n?","",text)
    text = re.sub(r"interview[s|ed|ing]","",text,re.IGNORECASE)
    text = re.sub(r".*track.*\n?","",text)
    text = re.sub(r".*available.*\n?","",text)
    text = re.sub(r".*Blog.*\n?","",text)
    text = re.sub(r".*Correction.*\n?","",text)
    text = re.sub(r".*Note.*\n?","",text)
    text = re.sub(r".*About.*\n?","",text)
    #text = re.sub(r".*YouTube.*.?","",text)
    text = re.sub(r".*lazy.*\n?","",text)
   
    #text = re.sub(r".*find.*.?","",text)
    text = re.sub(r".*Link[s].*\n?","",text)
    text = re.sub(r".*KaggleDaysMeetup.*\n?","",text)
    
    text = re.sub('\n', ' ', text)
    text = text.lower()
    text = re.sub(r"enjoy","",text)
    text = re.sub(r"show","",text)
    text = re.sub(r"even","",text)
    text = re.sub(r"another","",text)
    text = re.sub(r"time","",text)
    text = re.sub(r"part","",text)
    text = re.sub(r"sanyam","",text)
    text = re.sub(r"episode","",text)
    text = re.sub(r"interview","",text)
    text = re.sub(r"bhutani","",text)
    text = re.sub('\w*\d\w*', '', text)
    return text


def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    
    nopunc = clean_the_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    common_words = set(['sure','everyone','chai','us','also','hello','find','check','currently','two','description','welcome','podcast','many','like','get','lot','please','hi','heres','really','one'])
    stop_wds = common_words.union(sw.words('english'))
    remove_stopwords = [w for w in tokenized_text if w not in stop_wds]
    combined_text = ' '.join(remove_stopwords)
    return combined_text

In [ ]:
#preprocessing and counting word length of script and introduction
blog_details['script'] = blog_details['transcript'].apply(str).apply(lambda x: text_preprocessing(x))
blog_details['intro'] = blog_details['introduction'].apply(str).apply(lambda x: text_preprocessing(x))
blog_details['script_len'] = blog_details['script'].apply(str).apply(lambda x: len(x.split()))
blog_details['intro_len'] = blog_details['intro'].apply(str).apply(lambda x: len(x.split()))
blog_details

In [ ]:
#calculating the average word length of blog interviews and the range that they usually are in.
print("Average Word Length of blog interviews is {} words.".format(int(round(np.percentile(blog_details['script_len'],50)))))
lower_range = int(round(np.percentile(blog_details['script_len'],.1)))
upper_range = int(round(np.percentile(blog_details['script_len'],95)))
print("Word Length of blog interviews are between {} and {} words.".format(lower_range, upper_range))
print("Dr.Vladimir's interview is a long one (2564 words), which is standing below as an outlier.")
sns.set(style="whitegrid")
ax = sns.boxplot(x=blog_details["script_len"], linewidth=2.5)

In [ ]:
sorted_blog_details[['heroes','claps','script_len','intro_len']]

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.probability import FreqDist

# We will look at Frequency Distribution of words used in introduction
blog_guest_intro = []
for s in blog_details['intro']:
    blog_guest_intro.append(sent_tokenize(s))   

gi_string = ''.join(str(gi) for gi in blog_guest_intro)
gi_string = re.sub('[^a-zA-Z]', ' ', gi_string )
gi_tokenized_word = nltk.word_tokenize(gi_string)
gi_fdist = FreqDist(gi_tokenized_word)
print(gi_fdist.most_common(80))

%matplotlib inline
gi_fdist.plot(30, cumulative=False,title = "Common words used in Guest Introduction")
plt.show()

print("Bigrams :")
bigrams = nltk.bigrams(gi_tokenized_word)
print(list(bigrams)[:35])

# Please carry on your investigation